In [2]:
import torch as th
from peft import LoraConfig, PeftConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

bin c:\Users\su.osmond\AppData\Local\anaconda3\envs\llm-sft\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


Build a QLoRA LLM model based on the given HuggingFace name:

    1.   Create and prepare the bitsandbytes configuration for QLoRa's quantization
    2.   Download, load, and quantize on-the-fly Falcon-7b
    3.   Create and prepare the LoRa configuration
    4.   Load and configuration Falcon-7B's tokenizer

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=th.bfloat16,
)

In [ ]:
pretrained_model_path = "tiiuae/falcon-7b-instruct"
cache_dir = None

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_path,
    revision="main",
    quantization_config=bnb_config,
    load_in_4bit=True,
    device_map="auto", # {"": 0} to load entire model on GPU 0
    trust_remote_code=False,
    cache_dir=str(cache_dir) if cache_dir else None,
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_path,
    trust_remote_code=False,
    truncation=True,
    cache_dir=str(cache_dir) if cache_dir else None,
)

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": "<|pad|>"})
    with th.no_grad():
        model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json: 100%|██████████| 287/287 [00:00<?, ?B/s] 
c:\Users\su.osmond\AppData\Local\anaconda3\envs\llm-sft\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\su.osmond\.cache\huggingface\hub\models--tiiuae--falcon-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
tokenizer.json

NameError: name 'model' is not defined

In [7]:
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["query_key_value"],
)